# 프로젝트 - Movielens 영화 추천 실습  
- 이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작  
- 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터    
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로  
- MovieLens 1M Dataset 사용을 권장합니다.  
- 별점 데이터 implicit 데이터로 간주하고 테스트    
- 별점을 시청횟수로 해석  
- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외    
- MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운  

## Step 1. 데이터 준비와 전처리

Movielens 데이터  
rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터  

**`데이터 형식(head)`**  

**"ratings.dat" format:**    
`UserID::MovieID::Rating::Timestamp`  
**"users.dat" format:**  
`UserID::Gender::Age::Occupation::Zip-code`  
**"movies.dat"**    
`MovieID::Title::Genres`  

### 1) 라이브러리 import 및 데이터 가져오기¶

In [1]:
import os

import numpy as np
import pandas as pd
import re

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

In [2]:
#ratings 데이터 읽기
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')

orginal_data_size = len(ratings) #초기 data 수치
#print(orginal_data_size)
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [3]:
# 영화 제목을 보기 위해 메타 데이터를 읽기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python',  encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# 사용자 메타 데이터를 읽기
users_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/users.dat'
cols = ['user_id', 'gender', 'age', 'occupation', 'zip-code'] 
users = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python',  encoding = "ISO-8859-1")
users.head()

,user_id,gender,age,occupation,zip-code
0,1,Toy Story (1995),Animation|Children's|Comedy,NaN,NaN
1,2,Jumanji (1995),Adventure|Children's|Fantasy,NaN,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama,NaN,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN,NaN


In [5]:
# 3점 이상만 남김 (유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정)
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)    #3점 이상인 data 숫자

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')
#데이터 비율 확인

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [6]:
# rating 컬럼의 이름을 count로 바꿈(평점 데이터를 선호도로 사용하기 위해서)
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings.head(10)

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [7]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [8]:
#소문자로 바꾸기(# 검색을 쉽게하기 위해 제목 문자열을 소문자로 바꿈)
movies['title'] = movies['title'].str.lower() 
movies

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
...,...,...,...
3878,3948,meet the parents (2000),Comedy
3879,3949,requiem for a dream (2000),Drama
3880,3950,tigerland (2000),Drama
3881,3951,two family house (2000),Drama


## Step 2. 분석해 봅시다.

* ratings에 있는 유니크한 영화 개수  

In [9]:
ratings['movie_id'].nunique()

3628

* rating에 있는 유니크한 사용자 수  

In [10]:
ratings['user_id'].nunique()

6039

* 가장 인기 있는 영화 30개(인기순)  

In [11]:
#별점 정보와 영화 정보 필요하여 2가지 데이터 셋 합침

ratings_with_title = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings_with_title

mv_count = ratings_with_title.groupby('title')['user_id'].count()
mv_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

## Step 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [12]:
my_favorite = ['forrest gump (1994)' , 'godfather, the (1972)' ,'pulp fiction (1994)','sixth sense, the (1999)', 'silence of the lambs, the (1991)']

for movie in my_favorite:
    print(movies[movies['title'] == movie])
    print('--------------------------------------------------------------------')

     movie_id                title               genre
352       356  forrest gump (1994)  Comedy|Romance|War
--------------------------------------------------------------------
     movie_id                  title               genre
847       858  godfather, the (1972)  Action|Crime|Drama
--------------------------------------------------------------------
     movie_id                title        genre
293       296  pulp fiction (1994)  Crime|Drama
--------------------------------------------------------------------
      movie_id                    title     genre
2693      2762  sixth sense, the (1999)  Thriller
--------------------------------------------------------------------
     movie_id                             title           genre
589       593  silence of the lambs, the (1991)  Drama|Thriller
--------------------------------------------------------------------


In [13]:
my_mvid = [356, 858, 296, 2762, 593]

# 960917이라는 아이디 추가
my_movielist = pd.DataFrame({'user_id': ['999998']*5, 'count':[5]*5, 'movie_id': my_mvid})

if not ratings_with_title.isin({'user_id':[999998]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings_with_title = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings_with_title.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,999998,356,5,NaN
1,999998,858,5,NaN
2,999998,296,5,NaN
3,999998,2762,5,NaN
4,999998,593,5,NaN


In [14]:
# timestamp 컬럼 제거
ratings_with_title.drop('timestamp', axis=1, inplace=True)
ratings_with_title

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
0,999998,356,5
1,999998,858,5
2,999998,296,5
3,999998,2762,5


In [15]:
#user_id와 movie_id 고유한 값으로 만들기
#원래의 값과 고유한 값을 이어주는 dictionary를 생성

user_unique = ratings_with_title['user_id'].unique()
movie_unique = ratings_with_title['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

temp_user_data = ratings_with_title['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings_with_title):   
    print('user_id column indexing OK!!')
    ratings_with_title['user_id'] = temp_user_data    
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings_with_title['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings_with_title):
    print('movie column indexing OK!!')
    ratings_with_title['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings_with_title

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,160,5
1,6039,607,5
2,6039,222,5
3,6039,38,5


## Step 4. CSR matrix를 직접 만들어 봅시다.

In [16]:
num_user = ratings_with_title['user_id'].nunique()
num_movie = ratings_with_title['movie_id'].nunique()

csr_data = csr_matrix((ratings_with_title['count'], (ratings_with_title.user_id, ratings_with_title.movie_id)))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## Step 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [17]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
#Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [19]:
#als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose)¶
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
#model fitting
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## Step 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [21]:
limjs = user_to_idx['999998']

In [22]:
#영화 index  나의 선호도 파악
mo_in1 = movie_to_idx[1]
mo_in1_vector = als_model.item_factors[mo_in1]
np.dot(limjs, mo_in1_vector)

array([ -76.79098456, -165.5876094 ,  144.32125872,    6.01618127,
        -63.1284494 ,   64.71251014,   67.01919792,  -34.50772461,
         30.12946548,  -77.16757918,   88.24973334,  -54.94571785,
         11.64828062,   38.42286835,  125.86506806, -121.56542479,
        -27.64424691,   -3.01350258,   26.82118189,    6.19613711,
        -29.01389691, -152.10185589,   55.02359693,  -27.53655082,
        133.99397547,   60.3192455 , -157.41916347,   24.81082777,
         12.75675981,  -23.68047865,   36.49771329,  100.12762426,
         17.03833656,   43.74372764,  -77.98640914,   41.38326915,
         81.06402508,  222.0537664 ,  263.48328101,    4.49404254,
         37.98303739,   69.36783142,  212.11568173,  -51.11310164,
        -24.48322886,  -81.5370026 ,   61.90450983,  106.7680608 ,
        -29.61950566,  105.67122944,   97.9176287 ,  -73.71595577,
        219.53945354,   79.56831017,  -44.02982516, -143.38531486,
         92.24210657,  131.06539109,    8.13238672,  291.67826

In [23]:
#movie_index 858 gotfather와 비슷한 영화 찾기(15개)
movie_id = movie_to_idx[858]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(607, 0.99999994),
 (380, 0.9678637),
 (435, 0.55200404),
 (128, 0.4446847),
 (3517, 0.40910873),
 (3150, 0.36442578),
 (601, 0.35750768),
 (3531, 0.3549056),
 (3499, 0.34238124),
 (3601, 0.31595317),
 (3493, 0.31205475),
 (269, 0.31125468),
 (432, 0.30966806),
 (361, 0.30739793),
 (47, 0.29751965)]

## Step 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [24]:
def get_similar_movie(favorite_movie):
    movie_id = movie_to_idx[favorite_movie]
    similar_movie = als_model.similar_items(movie_id, N=15)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    recommand_list = [idx_to_movie[i[0]] for i in similar_movie]
    for movie in recommand_list:
        print(movies[movies['movie_id'] == movie], '--------------------------------------------------------------------', sep='\n')

get_similar_movie(858)

     movie_id                  title               genre
847       858  godfather, the (1972)  Action|Crime|Drama
--------------------------------------------------------------------
      movie_id                           title               genre
1203      1221  godfather: part ii, the (1974)  Action|Crime|Drama
--------------------------------------------------------------------
      movie_id                            title               genre
1954      2023  godfather: part iii, the (1990)  Action|Crime|Drama
--------------------------------------------------------------------
      movie_id                          title                        genre
1884      1953  french connection, the (1971)  Action|Crime|Drama|Thriller
--------------------------------------------------------------------
      movie_id                                              title        genre
1728      1787  paralyzing fear: the story of polio in america...  Documentary
--------------------------------

## Step 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [25]:
def recommand_movie(user_id):
    user = user_to_idx[user_id]
    movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    recommand_list = [idx_to_movie[i[0]] for i in movie_recommended]
    for movie in recommand_list:
        print(movies[movies['movie_id'] == movie], '--------------------------------------------------------------------', sep='\n')

recommand_movie('999998')   #limjs id : 99998

     movie_id         title                 genre
604       608  fargo (1996)  Crime|Drama|Thriller
--------------------------------------------------------------------
      movie_id                           title               genre
1203      1221  godfather: part ii, the (1974)  Action|Crime|Drama
--------------------------------------------------------------------
    movie_id                       title           genre
49        50  usual suspects, the (1995)  Crime|Thriller
--------------------------------------------------------------------
     movie_id                             title  genre
315       318  shawshank redemption, the (1994)  Drama
--------------------------------------------------------------------
      movie_id              title        genre
1195      1213  goodfellas (1990)  Crime|Drama
--------------------------------------------------------------------
     movie_id                    title      genre
523       527  schindler's list (1993)  Drama|War
---

In [30]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
idx_to_movie

{0: 1193,
 1: 661,
 2: 914,
 3: 3408,
 4: 2355,
 5: 1197,
 6: 1287,
 7: 2804,
 8: 594,
 9: 919,
 10: 595,
 11: 938,
 12: 2398,
 13: 2918,
 14: 1035,
 15: 2791,
 16: 2687,
 17: 2018,
 18: 3105,
 19: 2797,
 20: 2321,
 21: 720,
 22: 1270,
 23: 527,
 24: 2340,
 25: 48,
 26: 1097,
 27: 1721,
 28: 1545,
 29: 745,
 30: 2294,
 31: 3186,
 32: 1566,
 33: 588,
 34: 1907,
 35: 783,
 36: 1836,
 37: 1022,
 38: 2762,
 39: 150,
 40: 1,
 41: 1961,
 42: 1962,
 43: 2692,
 44: 260,
 45: 1028,
 46: 1029,
 47: 1207,
 48: 2028,
 49: 531,
 50: 3114,
 51: 608,
 52: 1246,
 53: 1357,
 54: 3068,
 55: 1537,
 56: 647,
 57: 2194,
 58: 648,
 59: 2268,
 60: 2628,
 61: 1103,
 62: 2916,
 63: 3468,
 64: 1210,
 65: 1792,
 66: 1687,
 67: 3578,
 68: 2881,
 69: 3030,
 70: 1217,
 71: 2126,
 72: 3108,
 73: 3035,
 74: 1253,
 75: 1610,
 76: 292,
 77: 2236,
 78: 3071,
 79: 368,
 80: 1259,
 81: 3147,
 82: 1544,
 83: 1293,
 84: 1188,
 85: 3255,
 86: 3257,
 87: 110,
 88: 2278,
 89: 2490,
 90: 1834,
 91: 3471,
 92: 589,
 93: 1690,
 9

In [33]:
movie_to_idx = {v:k for v,k in movie_to_idx.items()}
movie_to_idx

{1193: 0,
 661: 1,
 914: 2,
 3408: 3,
 2355: 4,
 1197: 5,
 1287: 6,
 2804: 7,
 594: 8,
 919: 9,
 595: 10,
 938: 11,
 2398: 12,
 2918: 13,
 1035: 14,
 2791: 15,
 2687: 16,
 2018: 17,
 3105: 18,
 2797: 19,
 2321: 20,
 720: 21,
 1270: 22,
 527: 23,
 2340: 24,
 48: 25,
 1097: 26,
 1721: 27,
 1545: 28,
 745: 29,
 2294: 30,
 3186: 31,
 1566: 32,
 588: 33,
 1907: 34,
 783: 35,
 1836: 36,
 1022: 37,
 2762: 38,
 150: 39,
 1: 40,
 1961: 41,
 1962: 42,
 2692: 43,
 260: 44,
 1028: 45,
 1029: 46,
 1207: 47,
 2028: 48,
 531: 49,
 3114: 50,
 608: 51,
 1246: 52,
 1357: 53,
 3068: 54,
 1537: 55,
 647: 56,
 2194: 57,
 648: 58,
 2268: 59,
 2628: 60,
 1103: 61,
 2916: 62,
 3468: 63,
 1210: 64,
 1792: 65,
 1687: 66,
 3578: 67,
 2881: 68,
 3030: 69,
 1217: 70,
 2126: 71,
 3108: 72,
 3035: 73,
 1253: 74,
 1610: 75,
 292: 76,
 2236: 77,
 3071: 78,
 368: 79,
 1259: 80,
 3147: 81,
 1544: 82,
 1293: 83,
 1188: 84,
 3255: 85,
 3257: 86,
 110: 87,
 2278: 88,
 2490: 89,
 1834: 90,
 3471: 91,
 589: 92,
 1690: 93,
 3

## ◈ 루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

평가문항/상세기준
1. CSR matrix가 정상적으로 만들어졌다.
사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.


## ◈ 회고

좋아하는 영화로 선정한 godfather과 유사도가 1.0인 결과를 확인해보니 대부가 1,2,3 으로 시리즈로 나왔기 때문에 똑같은 텍스트로 인식하여 유사도가 1로 나오는 결과를 초래한 것으로 보인다.
이런 문제를 보완하기 위해서는 자기 자신과 같은 이름의 영화 제목을 필터링 해 줘야 할 필요성을 느겼다. 

아직까지도 CSR matrix의 형태에 사용법에 대해서 잘 모르기 때문에 노드 진행과 포로젝트 진행에 어려움이 많았다. 
잘 되어진 코드를 보고 천천히 따라해 보면서 부족한 부분을 찾아가면서 다시 학습을 할 수 밖에 없었고 시간 투자에 비해서 잘 이해가 되지 않는 프로젝트였다. 